⚠️ API Key Requirement : 
> ->To run this code, you must have a valid **GEMINI_API_KEY**.  
> ->Visit [Google AI Studio](https://aistudio.google.com/app/apikey) to generate your API key.  
> ->Once obtained, specify the **Gemini model** you intend to use for reasoning tasks.


In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
import torch.nn as nn
import numpy as np
import os  # Added for environment variables if needed, though Kaggle Secrets is preferred

# --- Attempt to import Kaggle secrets and google generativeai ---
try:
    from kaggle_secrets import UserSecretsClient
    print("Kaggle UserSecretsClient imported successfully.")
except ImportError:
    UserSecretsClient = None
    print("Warning: Kaggle UserSecretsClient not found. API key needs to be handled differently if not in Kaggle.")

try:
    import google.generativeai as genai
    print("Google Generative AI SDK imported successfully.")
except ImportError:
    genai = None
    print("Error: Google Generative AI SDK not installed. Please install using: pip install google-generativeai")

# --- Configure Gemini API (needs to be done once) ---
GEMINI_API_KEY = None
gemini_model = None

if genai is not None:
    try:
        # Directly input your API key here
        GEMINI_API_KEY = "Your api key here"
        genai.configure(api_key=GEMINI_API_KEY)
        # Initialize the specific model we want to use
        gemini_model = genai.GenerativeModel('gemini-1.5-flash-latest')  # Or 'gemini-1.5-flash'
        print("Gemini API configured successfully with gemini-1.5-flash.")
    except Exception as e:
        print(f"Error configuring Gemini API: {e}. Please ensure GEMINI_API_KEY is set correctly in the code.")
else:
    print("Cannot configure Gemini API because the SDK is not installed.")

# --- GradientReversal and DATModel class definitions ---
class GradientReversal(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, lambda_):
        ctx.lambda_ = lambda_
        return x

    @staticmethod
    def backward(ctx, grad_output):
        return -ctx.lambda_ * grad_output, None

class DATModel(nn.Module):
    def __init__(self, base_model, hidden_size=768, num_classes=2, num_domains=2):  # Corrected __init__
        super(DATModel, self).__init__()
        self.base_model = base_model
        self.task_classifier = nn.Linear(hidden_size, num_classes)
        self.domain_classifier = nn.Linear(hidden_size, num_domains)
        self.dropout = nn.Dropout(0.1)

    # forward to handle output_attentions
    def forward(self, input_ids, attention_mask, lambda_=1.0, output_attentions=False):
        outputs = self.base_model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            output_attentions=output_attentions  # Pass the flag here
        )
        # features uses the [CLS] token representation (index 0)
        features = outputs.last_hidden_state[:, 0, :]
        features = self.dropout(features)
        task_logits = self.task_classifier(features)

        # Domain classification part 
        domain_features = GradientReversal.apply(features, lambda_)
        domain_logits = self.domain_classifier(domain_features)

        if output_attentions:
            # Return task logits, domain logits, and attentions
            return task_logits, domain_logits, outputs.attentions
        else:
            # Return only task and domain logits
            return task_logits, domain_logits

# --- Setup device, load tokenizer, base model ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model_loaded_successfully = False  # Flag to track model loading
tokenizer = None
model = None

try:
    tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")
    base_model = AutoModel.from_pretrained("google/muril-base-cased")

    # --- Create DATModel instance and load state dict ---
    model = DATModel(base_model).to(device)
    # Adjust path as needed
    state_dict_path = '/kaggle/input/dat_nlp_model1/pytorch/default/1/model_epoch_3.pth'  # Example path

    state_dict = torch.load(state_dict_path, map_location=device)
    # Remove 'module.' prefix if present
    if list(state_dict.keys())[0].startswith('module.'):
        state_dict = {k.replace('module.', ''): v for k, v in state_dict.items()}
    model.load_state_dict(state_dict)
    model.eval()  # Set model to evaluation mode
    print("MuRIL DATModel state dictionary loaded successfully.")
    model_loaded_successfully = True

except FileNotFoundError:
    print(f"Error: MuRIL Model state dictionary not found at {state_dict_path}. Ensure the path is correct.")
except Exception as e:
    print(f"An error occurred loading the MuRIL DATModel state dictionary: {e}")

# --- Function to get prediction and attentions ---
def get_prediction_and_attention(text, model, tokenizer, device):
    if not model_loaded_successfully or model is None or tokenizer is None:
         print("MuRIL Model or Tokenizer not loaded. Cannot get prediction.")
         return None, None, None

    model.eval()
    encoding = tokenizer(
        text,
        return_tensors='pt',
        max_length=128,  
        truncation=True,
        padding='max_length'
    )
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        # Request attentions from the model
        task_logits, _, attentions = model(
            input_ids,
            attention_mask,
            output_attentions=True  # IMPORTANT: Request attentions
        )

    _, predicted_class_idx = torch.max(task_logits, 1)
    prediction = "Suicidal" if predicted_class_idx.item() == 1 else "Non-Suicidal"

    # Return prediction, encoding dict, and attentions tuple
    return prediction, encoding, attentions

# --- Function to process attentions and identify important tokens ---
def get_important_tokens(encoding, attentions, tokenizer, top_n=5):
    """
    Processes attention weights to find the most attended-to tokens.
    This is a simplified approach averaging last layer attentions.
    """
    if encoding is None or attentions is None or tokenizer is None:
        print("Cannot get important tokens due to missing inputs.")
        return []

    try:
        input_ids = encoding['input_ids'][0]  # Get IDs for the first (only) batch item
        tokens = tokenizer.convert_ids_to_tokens(input_ids)

        # Attentions is a tuple of tensors, one per layer. Shape: (batch_size, num_heads, seq_len, seq_len)
        # We'll focus on the last layer's attentions
        last_layer_attentions = attentions[-1].cpu().numpy()  # Shape: (1, num_heads, seq_len, seq_len)

        # Average attention scores across all heads in the last layer
        avg_last_layer_attentions = np.mean(last_layer_attentions[0], axis=0)  # Shape: (seq_len, seq_len)

        # We want to know which input tokens received the most attention overall.
        # Sum attention scores directed *to* each token (column sums)
        token_attention_scores = np.sum(avg_last_layer_attentions, axis=0)  # Summing across rows (attention *from* all tokens *to* each token `j`)

        # Ignore special tokens like [CLS], [SEP], [PAD]
        special_tokens = {tokenizer.cls_token, tokenizer.sep_token, tokenizer.pad_token}
        token_scores = []
        for i, token in enumerate(tokens):
            # Ensure index is within bounds and token is not special/padding
            if i < len(token_attention_scores) and token not in special_tokens and input_ids[i] != tokenizer.pad_token_id:
                token_scores.append((token, token_attention_scores[i]))

        # Sort tokens by attention score in descending order
        token_scores.sort(key=lambda x: x[1], reverse=True)

        # Get the top N tokens (handle potential subword tokens)
        important_tokens = [token for token, score in token_scores[:top_n]]

        return important_tokens
    except Exception as e:
        print(f"Error processing attention weights: {e}")
        return []

# --- Function to generate explanation using Google Gemini API ---
def generate_explanation(text, prediction, important_tokens):
    """
    Generates an explanation by prompting the configured Google Gemini model.
    (Modified prompt to keep Telugu tokens in the output explanation)
    """
    # Check if Gemini model is configured and available
    if gemini_model is None:
        print("Gemini model not configured or API key missing. Returning placeholder explanation.")
        # Fallback placeholder explanation
        important_words_str_placeholder = " ".join(important_tokens).replace(" ##", "")
        return (
            f"(Placeholder - Gemini API not configured) The model predicted '{prediction}' likely because it focused on: '{important_words_str_placeholder}'. "
            f"These words might indicate '{prediction}' sentiment based on training data. "
            f"The model's attention highlights these as key terms."
        )

    # Combine tokens, potentially cleaning up subword indicators like '##'
    # For the prompt itself, joining with space is fine.
    important_words_str = " ".join(important_tokens) # Keep subword markers for the prompt list if present
   


    # ---PROMPT ---
 
    prompt = f"""The following Telugu text was analyzed by a text classification model (MuRIL-based):
Text: "{text}"

The model predicted the text as: {prediction}

Analysis of the model's internal attention mechanism shows it focused heavily on the following tokens/words: "{important_words_str}".

Based *only* on the text, the prediction ({prediction}), and the fact that the model focused on these specific words, provide a brief explanation (3-4 sentences in English) for why the *original model* likely arrived at its decision.

**Crucially, in your English explanation, when you mention the specific important words/tokens from the list provided ({important_words_str}), you MUST write those specific words using their original Telugu script.** For example, instead of saying '...focused on "last" and "day"...', you should say '...focused on "చివరి" and "రోజు"...'. Do not translate or transliterate these specific keywords within your explanation text itself.

Do not add your own assessment of the text's suicidality; explain the first model's *inferred reasoning* based on its focus points using the original Telugu keywords where appropriate.
"""
    # --- END OF PROMPT ---


    # --- LLM Interaction using Gemini API ---
    try:
      
        response = gemini_model.generate_content(
            prompt,
            
            )

        # Extract the text part of the response
        explanation = response.text
       
        return explanation

    except Exception as e:
        print(f"An error occurred during Gemini API call: {e}")
        # Return a fallback message indicating the error
        return f"(Error generating explanation via Gemini API: {e})"
    # --- End LLM Interaction ---


Kaggle UserSecretsClient imported successfully.
Google Generative AI SDK imported successfully.
Gemini API configured successfully with gemini-1.5-flash.
Using device: cuda


tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/953M [00:00<?, ?B/s]

<ipython-input-1-a8eae6212314>:98: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(state_dict_path, map_location=device)


MuRIL DATModel state dictionary loaded successfully.


**Model inference examples->**

**Example 1**

In [ ]:
if model_loaded_successfully and tokenizer is not None: # Proceed only if MuRIL model loaded

    telugu_text = "నా చివరి రోజు ఇది. ఎవరూ నన్ను ఆపకండి." 
    # This is my last day. No one should stop me.

    # 1. Get prediction and attentions from DATModel
    prediction, encoding, attentions = get_prediction_and_attention(telugu_text, model, tokenizer, device)

    if prediction is not None: # Check if prediction was successful
        # 2. Identify important tokens based on attention
        important_tokens = get_important_tokens(encoding, attentions, tokenizer, top_n=5)

        # 3. Generate the explanation using Gemini API
        # The function now directly uses the configured gemini_model
        explanation = generate_explanation(telugu_text, prediction, important_tokens)

        # 4. Print results
        print(f"\n--- Results for Input Telugu Text  ---")
        print(f"Input: {telugu_text}")
        print(f"Prediction (MuRIL): {prediction}") # Use the obtained prediction
        print(f"Identified Important Tokens (MuRIL Focus): {important_tokens}")
        print(f"\nExplanation (Generated by Gemini based on MuRIL focus):")
        print(explanation)
        print("---------------------------\n")


--- Results for Input Telugu Text  ---
Input: నా చివరి రోజు ఇది. ఎవరూ నన్ను ఆపకండి.
Prediction (MuRIL): Suicidal
Identified Important Tokens (MuRIL Focus): ['నా', '.', '##పకం', 'చివరి', 'రోజు']

Explanation (Generated by Gemini based on MuRIL focus):
The model likely classified the text as suicidal because of the phrase "నా చివరి రోజు," meaning "my last day."  The inclusion of  "ఎవరూ నన్ను ఆపకండి,"  ("Don't stop me"), further reinforces this interpretation. The model's attention to "##పకం" (part of "ఆపకండి") suggests it recognized the assertive and potentially desperate tone of refusal to be stopped.  The combination of "నా చివరి రోజు" and the refusal to be stopped led the model to predict a suicidal intent.

---------------------------



**Example 2**

In [ ]:
if model_loaded_successfully and tokenizer is not None: # Proceed only if MuRIL model loaded

    telugu_text = "సినిమా చాలా బాగుంది, కథ కూడా చక్కగా ఉంది." 
    # The movie was really good, and the story was nice too.

    # 1. Get prediction and attentions from  DATModel
    prediction, encoding, attentions = get_prediction_and_attention(telugu_text, model, tokenizer, device)

    if prediction is not None: # Check if prediction was successful
        # 2. Identify important tokens based on attention
        important_tokens = get_important_tokens(encoding, attentions, tokenizer, top_n=5)

        # 3. Generate the explanation using Gemini API
        # The function now directly uses the configured gemini_model
        explanation = generate_explanation(telugu_text, prediction, important_tokens)

        # 4. Print results
        print(f"\n--- Results for Input Telugu Text  ---")
        print(f"Input: {telugu_text}")
        print(f"Prediction (MuRIL): {prediction}") # Use the obtained prediction
        print(f"Identified Important Tokens (MuRIL Focus): {important_tokens}")
        print(f"\nExplanation (Generated by Gemini based on MuRIL focus):")
        print(explanation)
        print("---------------------------\n")


--- Results for Input Telugu Text  ---
Input: సినిమా చాలా బాగుంది, కథ కూడా చక్కగా ఉంది.
Prediction (MuRIL): Non-Suicidal
Identified Important Tokens (MuRIL Focus): ['సినిమా', '.', '##ా', 'బ', '##గు']

Explanation (Generated by Gemini based on MuRIL focus):
The model likely classified the text as "Non-Suicidal" because its attention mechanism prioritized positive sentiment words.  The focus on "సినిమా" (movie) and "బాగుంది" (good) suggests the model interpreted the overall tone as expressing enjoyment and satisfaction. The presence of  "చాలా" (very) amplifies the positive sentiment expressed towards the movie.  Therefore, the absence of negative or self-harming language, coupled with the focus on positive words like  "సినిమా" and "బాగుంది", led to the "Non-Suicidal" classification.

---------------------------



**Example 3**

In [ ]:
if model_loaded_successfully and tokenizer is not None: # Proceed only if MuRIL model loaded

    telugu_text =  "నా జీవితంలో ఏమీ సరిగ్గా జరగడం లేదు. ఎవరూ నన్ను అర్థం చేసుకోవడం లేదు, నా కష్టాలను ఎవరూ పట్టించుకోవడం లేదు. నాకేమీ ఇష్టం లేదు ఇక సజీవంగా ఉండటానికి." 
    # Nothing is going right in my life. No one understands me, and no one cares about my struggles. I have no interest in staying alive anymore

    # 1. Get prediction and attentions from  DATModel
    prediction, encoding, attentions = get_prediction_and_attention(telugu_text, model, tokenizer, device)

    if prediction is not None: # Check if prediction was successful
        # 2. Identify important tokens based on attention
        important_tokens = get_important_tokens(encoding, attentions, tokenizer, top_n=5)

        # 3. Generate the explanation using Gemini API
        # The function now directly uses the configured gemini_model
        explanation = generate_explanation(telugu_text, prediction, important_tokens)

        # 4. Print results
        print(f"\n--- Results for Input Telugu Text  ---")
        print(f"Input: {telugu_text}")
        print(f"Prediction (MuRIL): {prediction}") # Use the obtained prediction
        print(f"Identified Important Tokens (MuRIL Focus): {important_tokens}")
        print(f"\nExplanation (Generated by Gemini based on MuRIL focus):")
        print(explanation)
        print("---------------------------\n")


--- Results for Input Telugu Text  ---
Input: నా జీవితంలో ఏమీ సరిగ్గా జరగడం లేదు. ఎవరూ నన్ను అర్థం చేసుకోవడం లేదు, నా కష్టాలను ఎవరూ పట్టించుకోవడం లేదు. నాకేమీ ఇష్టం లేదు ఇక సజీవంగా ఉండటానికి.
Prediction (MuRIL): Suicidal
Identified Important Tokens (MuRIL Focus): ['##ష్టాలను', 'సరిగ్గా', 'క', '.', '##జీ']

Explanation (Generated by Gemini based on MuRIL focus):
The model likely classified the text as suicidal due to the negative sentiment expressed throughout.  The model's attention to "##ష్టాలను" (kashṭālanu - problems/difficulties) and "సరిగ్గా" (sarigga - correctly/properly) highlights the perceived lack of positive aspects in the speaker's life.  The focus on "##జీ" (jī - life), within the context of the overwhelmingly negative statements, likely reinforced the model's association with suicidal ideation.  The overall combination of these words, within the context of expressing unhappiness and hopelessness, led the model to its prediction.

---------------------------



**Example 4**

In [ ]:
if model_loaded_successfully and tokenizer is not None: # Proceed only if MuRIL model loaded

    telugu_text = "నాకు చదవడం అంటే చాలా ఇష్టం. కొత్త విషయాలు నేర్చుకోవడం నాకు ఆనందాన్ని ఇస్తుంది. ప్రతిరోజూ కాస్త కాస్తగా కొత్త విషయాలు తెలుసుకుంటూ ముందుకు సాగాలని అనుకుంటున్నాను." 
    # I love reading. Learning new things makes me happy. I want to keep learning little by little every day and move forward.

    # 1. Get prediction and attentions from  DATModel
    prediction, encoding, attentions = get_prediction_and_attention(telugu_text, model, tokenizer, device)

    if prediction is not None: # Check if prediction was successful
        # 2. Identify important tokens based on attention
        important_tokens = get_important_tokens(encoding, attentions, tokenizer, top_n=5)

        # 3. Generate the explanation using Gemini API
        # The function now directly uses the configured gemini_model
        explanation = generate_explanation(telugu_text, prediction, important_tokens)

        # 4. Print results
        print(f"\n--- Results for Input Telugu Text  ---")
        print(f"Input: {telugu_text}")
        print(f"Prediction (MuRIL): {prediction}") # Use the obtained prediction
        print(f"Identified Important Tokens (MuRIL Focus): {important_tokens}")
        print(f"\nExplanation (Generated by Gemini based on MuRIL focus):")
        print(explanation)
        print("---------------------------\n")


--- Results for Input Telugu Text  ---
Input: నాకు చదవడం అంటే చాలా ఇష్టం. కొత్త విషయాలు నేర్చుకోవడం నాకు ఆనందాన్ని ఇస్తుంది. ప్రతిరోజూ కాస్త కాస్తగా కొత్త విషయాలు తెలుసుకుంటూ ముందుకు సాగాలని అనుకుంటున్నాను.
Prediction (MuRIL): Non-Suicidal
Identified Important Tokens (MuRIL Focus): ['చదవడం', '.', 'చాలా', '.', 'అంటే']

Explanation (Generated by Gemini based on MuRIL focus):
The model likely classified the text as Non-Suicidal because the highlighted words "చదవడం", "చాలా", and "అంటే" suggest a positive sentiment.  The phrase indicates a strong liking ("చాలా ఇష్టం") for reading ("చదవడం"), expressing enjoyment and interest ("అంటే"). This positive sentiment, focusing on intellectual pursuits, likely led the model to disregard any potential indicators of suicidal ideation.  The model might have interpreted the overall tone as optimistic and goal-oriented.

---------------------------



**Example 5**

In [ ]:
if model_loaded_successfully and tokenizer is not None: # Proceed only if MuRIL model loaded

    telugu_text = "నా మీద వచ్చిన ఒత్తిడి, బాధ, అనారోగ్యం నాకు మరింత నష్టమే తెచ్చింది. జీవితంలో ఏమీ సరిగ్గా జరగడం లేదు. ఇక ఈ బాధను తట్టుకోలేను." 
    # The pressure, pain, and illness I am facing have only brought me more suffering. Nothing is going right in life. I can’t endure this pain anymore

    # 1. Get prediction and attentions from  DATModel
    prediction, encoding, attentions = get_prediction_and_attention(telugu_text, model, tokenizer, device)

    if prediction is not None: # Check if prediction was successful
        # 2. Identify important tokens based on attention
        important_tokens = get_important_tokens(encoding, attentions, tokenizer, top_n=5)

        # 3. Generate the explanation using Gemini API
        # The function now directly uses the configured gemini_model
        explanation = generate_explanation(telugu_text, prediction, important_tokens)

        # 4. Print results
        print(f"\n--- Results for Input Telugu Text  ---")
        print(f"Input: {telugu_text}")
        print(f"Prediction (MuRIL): {prediction}") # Use the obtained prediction
        print(f"Identified Important Tokens (MuRIL Focus): {important_tokens}")
        print(f"\nExplanation (Generated by Gemini based on MuRIL focus):")
        print(explanation)
        print("---------------------------\n")


--- Results for Input Telugu Text  ---
Input: నా మీద వచ్చిన ఒత్తిడి, బాధ, అనారోగ్యం నాకు మరింత నష్టమే తెచ్చింది. జీవితంలో ఏమీ సరిగ్గా జరగడం లేదు. ఇక ఈ బాధను తట్టుకోలేను.
Prediction (MuRIL): Suicidal
Identified Important Tokens (MuRIL Focus): ['ఒత్తిడి', 'మీద', 'వచ్చిన', 'బాధ', 'నా']

Explanation (Generated by Gemini based on MuRIL focus):
The model likely classified the text as suicidal because the highlighted words "ఒత్తిడి", "మీద వచ్చిన", "బాధ", and "నా" strongly suggest overwhelming stress and suffering.  The phrase implies the individual is experiencing intense pressure ("ఒత్తిడి") impacting them negatively ("మీద వచ్చిన"), resulting in significant distress ("బాధ"). The inclusion of "నా" (my/me) personalizes the suffering and emphasizes the individual's feeling of being burdened.  The combination of these words created a pattern recognized by the model as indicative of suicidal ideation.

---------------------------



**Example 6**

In [ ]:
if model_loaded_successfully and tokenizer is not None: # Proceed only if MuRIL model loaded

    telugu_text = "ఈ రోజు చాలా బిజీగా గడిచింది. ఉదయం నుండి పని లోనే ఉన్నాను, కానీ సాయంత్రం వేళ కాఫీ తాగుతూ విశ్రాంతి తీసుకోవడం ఆనందంగా ఉంది." 
    # Today was very busy. I was working since morning, but in the evening, having coffee and relaxing felt good.

    # 1. Get prediction and attentions from  DATModel
    prediction, encoding, attentions = get_prediction_and_attention(telugu_text, model, tokenizer, device)

    if prediction is not None: # Check if prediction was successful
        # 2. Identify important tokens based on attention
        important_tokens = get_important_tokens(encoding, attentions, tokenizer, top_n=5)

        # 3. Generate the explanation using Gemini API
        # The function now directly uses the configured gemini_model
        explanation = generate_explanation(telugu_text, prediction, important_tokens)

        # 4. Print results
        print(f"\n--- Results for Input Telugu Text  ---")
        print(f"Input: {telugu_text}")
        print(f"Prediction (MuRIL): {prediction}") # Use the obtained prediction
        print(f"Identified Important Tokens (MuRIL Focus): {important_tokens}")
        print(f"\nExplanation (Generated by Gemini based on MuRIL focus):")
        print(explanation)
        print("---------------------------\n")


--- Results for Input Telugu Text  ---
Input: ఈ రోజు చాలా బిజీగా గడిచింది. ఉదయం నుండి పని లోనే ఉన్నాను, కానీ సాయంత్రం వేళ కాఫీ తాగుతూ విశ్రాంతి తీసుకోవడం ఆనందంగా ఉంది.
Prediction (MuRIL): Non-Suicidal
Identified Important Tokens (MuRIL Focus): ['.', 'ఈ', 'చాలా', 'రోజు', '##జీ']

Explanation (Generated by Gemini based on MuRIL focus):
The model likely classified the text as Non-Suicidal because it identified positive sentiments within the text.  The focus on "ఈ రోజు" (this day) and "చాలా" (very) suggests the model processed the overall context of the day, noting it was "బిజీ" (busy).  The attention given to "##జీ" (likely part of "బిజీ," meaning busy), combined with the description of a relaxing evening, likely led the model to conclude that despite a busy day ("ఈ రోజు"), there was a positive concluding element, suggesting a non-suicidal state.

---------------------------



**Example 7**

In [ ]:
if model_loaded_successfully and tokenizer is not None: # Proceed only if MuRIL model loaded

    telugu_text =  "అందరూ నన్ను వదిలేసారు నాకు ఒంటరితనం తప్ప మరేమీ లేదు ఇక ఈ బాధలో ఉండలేను." 
    # Everyone has left me. I have nothing but loneliness. I can’t stay in this pain any longer.

    # 1. Get prediction and attentions from  DATModel
    prediction, encoding, attentions = get_prediction_and_attention(telugu_text, model, tokenizer, device)

    if prediction is not None: # Check if prediction was successful
        # 2. Identify important tokens based on attention
        important_tokens = get_important_tokens(encoding, attentions, tokenizer, top_n=5)

        # 3. Generate the explanation using Gemini API
        # The function now directly uses the configured gemini_model
        explanation = generate_explanation(telugu_text, prediction, important_tokens)

        # 4. Print results
        print(f"\n--- Results for Input Telugu Text  ---")
        print(f"Input: {telugu_text}")
        print(f"Prediction (MuRIL): {prediction}") # Use the obtained prediction
        print(f"Identified Important Tokens (MuRIL Focus): {important_tokens}")
        print(f"\nExplanation (Generated by Gemini based on MuRIL focus):")
        print(explanation)
        print("---------------------------\n")


--- Results for Input Telugu Text  ---
Input: అందరూ నన్ను వదిలేసారు నాకు ఒంటరితనం తప్ప మరేమీ లేదు ఇక ఈ బాధలో ఉండలేను.
Prediction (MuRIL): Suicidal
Identified Important Tokens (MuRIL Focus): ['.', '##సారు', '##ది', 'వ', 'అందరూ']

Explanation (Generated by Gemini based on MuRIL focus):
The model likely classified the text as suicidal because of the phrases expressing feelings of abandonment and hopelessness.  The attention on "అందరూ" (everyone) and "##సారు" (left/abandoned), suggests the model recognized the phrase indicating social isolation. The presence of "##ది" (this) likely helped contextualize the following words relating to unbearable suffering, while "వదిలేసారు" (left/abandoned) reinforces the sentiment of desertion.  The model's focus on these words points to an interpretation of overwhelming loneliness and despair as expressed in the text.

---------------------------



**Example 8**

In [ ]:
if model_loaded_successfully and tokenizer is not None: # Proceed only if MuRIL model loaded

    telugu_text = "నాకు కొత్త ఉద్యోగం వచ్చింది చాలా సంతోషంగా ఉంది కుటుంబంతో సెలవు ప్లాన్ చేస్తున్నాను." 
    # I got a new job and I'm very happy. I'm planning a vacation with my family.

    # 1. Get prediction and attentions from  DATModel
    prediction, encoding, attentions = get_prediction_and_attention(telugu_text, model, tokenizer, device)

    if prediction is not None: # Check if prediction was successful
        # 2. Identify important tokens based on attention
        important_tokens = get_important_tokens(encoding, attentions, tokenizer, top_n=5)

        # 3. Generate the explanation using Gemini API
        # The function now directly uses the configured gemini_model
        explanation = generate_explanation(telugu_text, prediction, important_tokens)

        # 4. Print results
        print(f"\n--- Results for Input Telugu Text  ---")
        print(f"Input: {telugu_text}")
        print(f"Prediction (MuRIL): {prediction}") # Use the obtained prediction
        print(f"Identified Important Tokens (MuRIL Focus): {important_tokens}")
        print(f"\nExplanation (Generated by Gemini based on MuRIL focus):")
        print(explanation)
        print("---------------------------\n")


--- Results for Input Telugu Text  ---
Input: నాకు కొత్త ఉద్యోగం వచ్చింది చాలా సంతోషంగా ఉంది కుటుంబంతో సెలవు ప్లాన్ చేస్తున్నాను.
Prediction (MuRIL): Non-Suicidal
Identified Important Tokens (MuRIL Focus): ['ఉద్యోగం', 'వచ్చింది', 'కొత్త', '.', 'నాకు']

Explanation (Generated by Gemini based on MuRIL focus):
The model likely classified the text as Non-Suicidal because the sentence expresses positive sentiment.  The model's attention to "నాకు కొత్త ఉద్యోగం వచ్చింది" suggests it recognized the positive news of a new job as a strong indicator of well-being. The presence of  "నాకు" further reinforces a sense of agency and positive self-perception. The overall tone conveyed by these words strongly counteracts any indication of suicidal ideation.

---------------------------



**Example 9**

In [ ]:
if model_loaded_successfully and tokenizer is not None: # Proceed only if MuRIL model loaded

    telugu_text = "నన్ను అందరూ వదిలేసారు నాకు ఈ ఒంటరితనం తప్ప మరేమీ కనిపించడం లేదు ఈ బాధ నుండి బయటపడాలంటే చనిపోవడం తప్ప మరో మార్గం లేదని నమ్ముతున్నాను." 
    # Everyone has abandoned me, and I see nothing but loneliness. I believe that the only way to escape this pain is to end my life.

    # 1. Get prediction and attentions from  DATModel
    prediction, encoding, attentions = get_prediction_and_attention(telugu_text, model, tokenizer, device)

    if prediction is not None: # Check if prediction was successful
        # 2. Identify important tokens based on attention
        important_tokens = get_important_tokens(encoding, attentions, tokenizer, top_n=5)

        # 3. Generate the explanation using Gemini API
        # The function now directly uses the configured gemini_model
        explanation = generate_explanation(telugu_text, prediction, important_tokens)

        # 4. Print results
        print(f"\n--- Results for Input Telugu Text  ---")
        print(f"Input: {telugu_text}")
        print(f"Prediction (MuRIL): {prediction}") # Use the obtained prediction
        print(f"Identified Important Tokens (MuRIL Focus): {important_tokens}")
        print(f"\nExplanation (Generated by Gemini based on MuRIL focus):")
        print(explanation)
        print("---------------------------\n")


--- Results for Input Telugu Text  ---
Input: నన్ను అందరూ వదిలేసారు నాకు ఈ ఒంటరితనం తప్ప మరేమీ కనిపించడం లేదు ఈ బాధ నుండి బయటపడాలంటే చనిపోవడం తప్ప మరో మార్గం లేదని నమ్ముతున్నాను.
Prediction (MuRIL): Suicidal
Identified Important Tokens (MuRIL Focus): ['##ప', '##డా', '##ని', '##తున్న', '.']

Explanation (Generated by Gemini based on MuRIL focus):
The model likely classified the text as "Suicidal" because the phrase containing "పోవడం" (pōvaḍaṁ) and "తున్నాను" (tuṇṇānu) strongly suggests a belief in death as the only solution.  The model's attention on these words, along with the overall negative sentiment of the sentence, indicates it interpreted the statement as an expression of suicidal ideation. The presence of  "నమ్ముతున్నాను" (nammutunnaanu) further reinforces this interpretation, showing a conviction in this belief.

---------------------------



**Example 10**

In [ ]:
if model_loaded_successfully and tokenizer is not None: # Proceed only if MuRIL model loaded

    telugu_text = "నాకు ఈ మధ్య కొత్త ఉద్యోగం వచ్చింది దానికి నేను చాలా సంతోషంగా ఉన్నాను కుటుంబంతో కలిసి వచ్చే వారం సెలవుకి వెళ్లాలని ఆలోచిస్తూ టికెట్లు బుక్ చేస్తున్నాను." 
    # I recently got a new job, and I am very happy about it. I am planning to go on vacation with my family next week and am booking the tickets.

    # 1. Get prediction and attentions from  DATModel
    prediction, encoding, attentions = get_prediction_and_attention(telugu_text, model, tokenizer, device)

    if prediction is not None: # Check if prediction was successful
        # 2. Identify important tokens based on attention
        important_tokens = get_important_tokens(encoding, attentions, tokenizer, top_n=5)

        # 3. Generate the explanation using Gemini API
        # The function now directly uses the configured gemini_model
        explanation = generate_explanation(telugu_text, prediction, important_tokens)

        # 4. Print results
        print(f"\n--- Results for Input Telugu Text  ---")
        print(f"Input: {telugu_text}")
        print(f"Prediction (MuRIL): {prediction}") # Use the obtained prediction
        print(f"Identified Important Tokens (MuRIL Focus): {important_tokens}")
        print(f"\nExplanation (Generated by Gemini based on MuRIL focus):")
        print(explanation)
        print("---------------------------\n")


--- Results for Input Telugu Text  ---
Input: నాకు ఈ మధ్య కొత్త ఉద్యోగం వచ్చింది దానికి నేను చాలా సంతోషంగా ఉన్నాను కుటుంబంతో కలిసి వచ్చే వారం సెలవుకి వెళ్లాలని ఆలోచిస్తూ టికెట్లు బుక్ చేస్తున్నాను.
Prediction (MuRIL): Non-Suicidal
Identified Important Tokens (MuRIL Focus): ['సెలవు', 'ఉద్యోగం', '##ె', '##కి', 'వ']

Explanation (Generated by Gemini based on MuRIL focus):
The model likely classified the text as Non-Suicidal because the highlighted words "సెలవు" (leave/vacation), "ఉద్యోగం" (job), and "##ె ##కి వ" (likely part of a verb indicating future plans/actions) suggest positive life events.  The mention of a new "ఉద్యోగం" and upcoming "సెలవు" with family implies happiness and future plans, which are generally associated with a lack of suicidal ideation. The model's focus on the "##ె ##కి వ" sequence possibly indicates its recognition of the future tense, further strengthening the positive outlook.

---------------------------



**Example 11**

In [ ]:
if model_loaded_successfully and tokenizer is not None: # Proceed only if MuRIL model loaded

    telugu_text = "ఇటీవల నాకు కొత్త ఉద్యోగం లభించింది, అందుకు నేను ఎంతో ఆనందంగా ఉన్నాను, కానీ నాకు కుటుంబం లేదు, ఇది కొన్ని సందర్భాల్లో నిరాశాజనకంగా మరియు ఒంటరిగా అనిపిస్తుంది." 
    # Recently, I got a new job, and I am very happy about it, but I don't have a family, which feels disappointing and lonely at times.

    # 1. Get prediction and attentions from  DATModel
    prediction, encoding, attentions = get_prediction_and_attention(telugu_text, model, tokenizer, device)

    if prediction is not None: # Check if prediction was successful
        # 2. Identify important tokens based on attention
        important_tokens = get_important_tokens(encoding, attentions, tokenizer, top_n=5)

        # 3. Generate the explanation using Gemini API
        # The function now directly uses the configured gemini_model
        explanation = generate_explanation(telugu_text, prediction, important_tokens)

        # 4. Print results
        print(f"\n--- Results for Input Telugu Text  ---")
        print(f"Input: {telugu_text}")
        print(f"Prediction (MuRIL): {prediction}") # Use the obtained prediction
        print(f"Identified Important Tokens (MuRIL Focus): {important_tokens}")
        print(f"\nExplanation (Generated by Gemini based on MuRIL focus):")
        print(explanation)
        print("---------------------------\n")

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.



--- Results for Input Telugu Text  ---
Input: ఇటీవల నాకు కొత్త ఉద్యోగం లభించింది, అందుకు నేను ఎంతో ఆనందంగా ఉన్నాను, కానీ నాకు కుటుంబం లేదు, ఇది కొన్ని సందర్భాల్లో నిరాశాజనకంగా మరియు ఒంటరిగా అనిపిస్తుంది.
Prediction (MuRIL): Suicidal
Identified Important Tokens (MuRIL Focus): ['ఇటీవల', 'అందుకు', ',', 'కొత్త', 'ఉద్యోగం']

Explanation (Generated by Gemini based on MuRIL focus):
The model's prediction of "Suicidal" likely stems from a misinterpretation of the context.  The model may have focused on "ఇటీవల కొత్త ఉద్యోగం" ("recently new job"), interpreting a significant life change as potentially overwhelming or isolating. The inclusion of "అందుకు" ("for that/because of that") might have further reinforced this interpretation, suggesting the new job, rather than alleviating loneliness, is somehow contributing to the negative feelings expressed later in the text. The model might have failed to adequately weigh the explicit mention of loneliness and sadness against the seemingly positive cont

**Example 12**

In [ ]:
if model_loaded_successfully and tokenizer is not None: # Proceed only if MuRIL model loaded

    telugu_text = "నేను బ్రతకాలని అనుకోవడం లేదు, కానీ నా కుటుంబం కోసం బ్రతుకుతున్నాను." 
    # I don't want to live, but I am living for my family

    # 1. Get prediction and attentions from  DATModel
    prediction, encoding, attentions = get_prediction_and_attention(telugu_text, model, tokenizer, device)

    if prediction is not None: # Check if prediction was successful
        # 2. Identify important tokens based on attention
        important_tokens = get_important_tokens(encoding, attentions, tokenizer, top_n=5)

        # 3. Generate the explanation using Gemini API
        # The function now directly uses the configured gemini_model
        explanation = generate_explanation(telugu_text, prediction, important_tokens)

        # 4. Print results
        print(f"\n--- Results for Input Telugu Text  ---")
        print(f"Input: {telugu_text}")
        print(f"Prediction (MuRIL): {prediction}") # Use the obtained prediction
        print(f"Identified Important Tokens (MuRIL Focus): {important_tokens}")
        print(f"\nExplanation (Generated by Gemini based on MuRIL focus):")
        print(explanation)
        print("---------------------------\n")

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.



--- Results for Input Telugu Text  ---
Input: నేను బ్రతకాలని అనుకోవడం లేదు, కానీ నా కుటుంబం కోసం బ్రతుకుతున్నాను.
Prediction (MuRIL): Suicidal
Identified Important Tokens (MuRIL Focus): ['.', '##తున్న', ',', 'కానీ', '##ుకు']

Explanation (Generated by Gemini based on MuRIL focus):
The model likely classified the text as "Suicidal" because of the phrase "నేను బ్రతకాలని అనుకోవడం లేదు," which expresses a lack of desire to live.  The model's attention on "##తున్న" and "కానీ ##ుకు" suggests it interpreted the contrasting phrase "కానీ నా కుటుంబం కోసం బ్రతుకుతున్నాను" as a reason for continued living despite the initial expression of not wanting to live, highlighting the conflict and potentially associating it with suicidal ideation.  The model may have focused on these specific parts to identify the internal conflict and the statement implying lack of will to live.

---------------------------

